# Wpływ cech czerwonego wina na jego jakość - prezentacja wytrenowanych modeli
## Autorzy: Jakub Kosterna, Bartosz Siński, Jan Smoleń

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle
np.random.seed = 42

### Załadowanie zbioru danych i podział na zbiory treningowe i testowe 

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
df_wines = pd.read_csv('./src/winequality-red.csv')
df_wines["is_good"] = df_wines.apply(lambda row: 1 if row.quality > 5 else 0, axis = 1)
X=df_wines.drop(["is_good", 'quality'], axis=1)
y=df_wines["is_good"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state = 42)

### Base SVM

In [4]:
from sklearn.svm import SVC
svm_base=SVC(random_state=42)
svm_base.fit(X_train, y_train)
preds=svm_base.predict(X_test)

In [5]:
from sklearn.metrics import accuracy_score
accuracy_score(preds,y_test)

0.6375

Wynik surowego SVM na naszym zbiorze danych to około 75%. Tera poszukamy kombinacji parametrów dających najlepsze wyniki używając narzędzia `GridSearchCV`.

### Tuning hiperparametrów dla SVM

In [6]:
from sklearn.model_selection import GridSearchCV
cpar=[]
gpar=[]
for i in range(-4, 5):
    cpar.append(10**i)
for i in range(-4, 5):
    gpar.append(10**i)
gpar.append("auto")
gpar.append("scale")
params = [{'C': cpar,
        'kernel': ["rbf"], # "linear", "poly"],    bardzo długi czas wykonywania
        'gamma': gpar}]
svm_tuned=SVC(random_state=42)
gs_svm=GridSearchCV(svm_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
gs_svm.fit(X_train, y_train)
gs_svm.best_params_

{'C': 10000, 'gamma': 0.0001, 'kernel': 'rbf'}

In [7]:
accuracy_score(gs_svm.predict(X_test),y_test)

0.7475

Po dosyć długim przetestowaniu różnych kombinacji hiperparametrów używając narzędzia GridSearchCV, najlepszy wynik uzyskaliśmy dla {'C': 10000, 'gamma': 0.0001, 'kernel': 'rbf'}. Sprawdziliśmy następnie jeszcze wartości w okolicach tych parametrów.

In [8]:
cpar=[]
gpar=[]
for i in range(1, 11):
    cpar.append(round((10**3)*2*i, 8))
for i in range(1, 11):
    gpar.append(round((10**-4)*2*i, 8))
params = [{'C': cpar,
         'kernel': ["rbf"],
         'gamma': gpar}]
svm_tuned=SVC(random_state=42)
gs_svm=GridSearchCV(svm_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
gs_svm.fit(X_train, y_train)
gs_svm.best_params_   

{'C': 2000, 'gamma': 0.0006, 'kernel': 'rbf'}

In [9]:
svcr = gs_svm.predict(X_test)
accuracy_score(gs_svm.predict(X_test),y_test)

0.7525

Udało się jeszcze trochę polepszyć wyniki naszego modelu przyjmując parametry `{'C': 2000, 'gamma': 0.0006, 'kernel': 'rbf'}`, osiągając skuteczność powyżej 75%. Nie są to jednak duże różnice.

Używając SVM nie udało się siągnąć wyniku tak dobrego jak w przypadku np. xgboosta. Powodem tego jest zapewne spore nakładanie się na siebie naszych dwóch klas - w końcu granica pomiędzy nimi to różnica między subiektywną oceną 5 a 6 w 10 stopniowej skali, a SVM nie radzi sobie szczególnie dobrze z takimi problemami. Jest on też dosyć wolny - testowanie różnych konfiguracji parametrów, szczególnie przy użyciu opcji `kernel=poly`, zajmowało godziny.

### Random Forest z domyślnymi parametrami 

In [9]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1613)
rfc.fit(X_train,y_train)

RandomForestClassifier(random_state=1613)

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(rfc.predict(X_test),y_test)

0.7875

Klasyfikator Random Forest z ustawionymi domyslnymi parametrami poradził sobie lepiej na naszym zbiorze danych niż SVM i jednocześnie gorzej niż XBoost. Zobaczmy jak sytuacja będzie wyglądać po tuningu hiperparametrów naszego Random Forest.

### Random Forest z tuningiem hiperparametrów

In [32]:
from sklearn.model_selection import RandomizedSearchCV
rfc2 = RandomForestClassifier(random_state=1613)
grid = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
rfct = RandomizedSearchCV(estimator = rfc2,param_distributions = grid, cv=4,n_iter=10, random_state=42)
rfct.fit(X_train,y_train)
accuracy_score(rfct.predict(X_test),y_test)

0.7925

In [19]:
rfct.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 80,
 'bootstrap': True}

Udało nam się poprawić accuracy o 0,5% . Teraz poszukamy parametrów w otoczeniu tych, które poprawiają predykcje naszego modelu. Użyjemy do tego GridSearchCV, który zamiast wybierać próbki z rozkładów parametrów, bierze wszystkie kombinacje podanych mu parametrów.

In [23]:
from sklearn.model_selection import GridSearchCV
rfc2 = RandomForestClassifier(random_state=1613)
grid2 = {'bootstrap': [True],
 'max_depth': [ 75, 80, 85,90],
 'max_features': ['auto'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 3, 4],
 'n_estimators': [ 900,950, 1000,1050,1100]}
rfcg = GridSearchCV(estimator = rfc2, param_grid = grid2, cv = 4, n_jobs = -1, verbose = 2)
rfcg.fit(X_train,y_train)
accuracy_score(rfcg.predict(X_test),y_test)

Fitting 4 folds for each of 180 candidates, totalling 720 fits


0.795

In [28]:
rf = rfcg.predict(X_test)
rfcg.best_params_

{'bootstrap': True,
 'max_depth': 75,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 1050}

Jak widzimy udało się jeszcze podnieść accuracy jeszcze  o 0,25%. 

### XGBoost bez poszukiwań dobrych parametrów

In [11]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective = "binary:logistic", seed = 1613, use_label_encoder=False)
xgb_model.fit(X_train, y_train)
predsxg = xgb_model.predict(X_test)

[22:39:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [12]:
accuracy_score(predsxg,y_test)

0.8

Nawet bez boosting XGBoost wykazuje się najlepszym accuracy sposród wszystkich badanych przez nas modeli, chociaż jest on bardzo blisko Random Forest z dobranymi hiperparametrami. Spójrzmy jaki będzie rezultat z tuningiem hiperparametrów.

### XGBoost z tuningiem hiperparametrów

In [13]:
from sklearn.model_selection import RandomizedSearchCV
gbm_param_grid = {
    "learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30] ,
    "max_depth": [ 3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight": [ 1, 3, 5, 7],
    "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4],
    "colsample_bytree": [ 0.3, 0.4, 0.5 , 0.7]
}

gbm = xgb.XGBClassifier(objective = "binary:logistic", eval_metric = "logloss", use_label_encoder = False, seed = 42)

randomized_mse = RandomizedSearchCV(param_distributions = gbm_param_grid,
    estimator = gbm,
    cv = 4,
    n_iter = 1000)

randomized_mse.fit(X_train, y_train)
randomized_mse.best_params_

{'min_child_weight': 1,
 'max_depth': 10,
 'learning_rate': 0.05,
 'gamma': 0.0,
 'colsample_bytree': 0.4}

In [14]:
predsxg2 = randomized_mse.predict(X_test)
accuracy_score(predsxg2,y_test)

0.81

Dobór parametrów poprawił accuracy naszego XGBoosta o 1% co jest zdecydowanie najlepszym uzyskanym przez nas wynikiem. Powyżej badalismy jedynie metrykę accuracy, zobaczmy jak predykcje modeli różnią się także przy zastosowaniu innych metryk.

### Ocena modeli

In [27]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

results = {
    "algorithm" : ['SVC','Random Forest','XGBoost'],
    "accuracy" : [accuracy_score(y_test,svcr),accuracy_score(y_test,rf),accuracy_score(y_test,predsxg2)],
    "precision" : [precision_score(y_test,svcr),precision_score(y_test,rf),precision_score(y_test,predsxg2)],
    "recall" :[recall_score(y_test,svcr),recall_score(y_test,rf),recall_score(y_test,predsxg2)],
    'ROC AUC' : [roc_auc_score(y_test,svcr),roc_auc_score(y_test,rf),roc_auc_score(y_test,predsxg2)]
}
pd.DataFrame(results)

,algorithm,accuracy,precision,recall,ROC AUC
0,SVC,0.7525,0.791878,0.728972,0.754271
1,Random Forest,0.7950,0.805556,0.813084,0.793639
2,XGBoost,0.8100,0.819444,0.827103,0.808713


Jak widzimy model oparty na SVC miał precyzje zbliżoną do Random Forrest i XGBoost jednak bardzo odstający *recall*. Różnice wartość ROC AUC między modelami są takie same jak dla metryki accuracy. Dodatkowo spójrzmy na *table of confusion* dla tych modeli.

### SVC

In [29]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, svcr).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

,Actual positives,Actual negatives
Positive predictions,156,58
Negative predictions,41,145


### Random Forest

In [30]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, rf).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

,Actual positives,Actual negatives
Positive predictions,174,40
Negative predictions,42,144


### XGBoost

In [31]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, predsxg2).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

,Actual positives,Actual negatives
Positive predictions,177,37
Negative predictions,39,147
